In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
from keras.preprocessing import image                  
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

Ten classes: `c0`, `c1`, `c2`, `c3`, `c4`, `c5`, `c6`, `c7`, `c8`, `c9`. 

In [4]:
# Defining a function to load datasets.
def load_dataset(path, num_classes):
    data = load_files(path)
    data_files = np.array(data['filenames'])
    data_targets = np_utils.to_categorical(np.array(data['target']), num_classes= num_classes)
    return data_files, data_targets

In [5]:
# Load train dataset
train_files, train_targets = load_dataset('../../../../capstone-project-data/imgs/train', 10)

#valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('imgs_test')

In [ ]:
# Creating a validation set
train_files, valid_files, train_targets, valid_targets = train_test_split(train_files, train_targets,
                                                                          test_size=0.2, random_state=12)

In [6]:
# Take a look at the output of the load_dataset function
print(train_files[0])
print(train_targets[0])

../../../../capstone-project-data/imgs/train/c9/img_54.jpg
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [7]:
print(train_files[1])
print(train_targets[1])

../../../../capstone-project-data/imgs/train/c1/img_94750.jpg
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
print(train_files[2])
print(train_targets[2])

../../../../capstone-project-data/imgs/train/c9/img_90465.jpg
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [9]:
# Load list of driver actions
driver_actions = [item[-3:-1] for item in sorted(glob("../../../../capstone-project-data/imgs/train/*/"))]

In [10]:
driver_actions

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [11]:
print('There are %d training images.' % len(train_files))
print('There are %d possible driver action categories.' % len(driver_actions))

There are 22424 training images.
There are 10 possible driver action categories.


In [12]:
train_files

array(['../../../../capstone-project-data/imgs/train/c9/img_54.jpg',
       '../../../../capstone-project-data/imgs/train/c1/img_94750.jpg',
       '../../../../capstone-project-data/imgs/train/c9/img_90465.jpg',
       ...,
       '../../../../capstone-project-data/imgs/train/c4/img_2632.jpg',
       '../../../../capstone-project-data/imgs/train/c4/img_62157.jpg',
       '../../../../capstone-project-data/imgs/train/c1/img_18002.jpg'],
      dtype='<U62')

In [13]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(640, 480))
    # convert PIL.Image.Image type to 3D tensor with shape (640, 480, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 640, 480, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [14]:
def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255


 17%|█▋        | 3776/22424 [00:42<03:30, 88.55it/s]


 19%|█▊        | 4203/22424 [02:12<09:35, 31.64it/s]


 20%|██        | 4588/22424 [03:48<14:49, 20.06it/s]


 21%|██        | 4676/22424 [37:57<2:24:05,  2.05it/s]

In [21]:
train_tensors.max()

255.0